## Cleans TRACE Corporate Bond Data

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Loads in subset of prof Yoshio's data
with pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv", chunksize=1000) as reader:
    data = reader.get_chunk()

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cusip_id                1000 non-null   object 
 1   date                    1000 non-null   int64  
 2   price                   1000 non-null   float64
 3   volume                  1000 non-null   int64  
 4   ISSUE_ID                1000 non-null   int64  
 5   ISSUER_ID               1000 non-null   int64  
 6   PROSPECTUS_ISSUER_NAME  1000 non-null   object 
 7   MATURITY                1000 non-null   int64  
 8   SECURITY_LEVEL          1000 non-null   object 
 9   OFFERING_AMT            1000 non-null   int64  
 10  OFFERING_DATE           1000 non-null   int64  
 11  OFFERING_PRICE          852 non-null    float64
 12  INTEREST_FREQUENCY      1000 non-null   int64  
 13  COUPON                  1000 non-null   float64
 14  standard                1000 non-null   i

In [174]:
# data.memory_usage(deep=True) \
# .gt(8000)

As expected all "object" (ie. string) columns use up the majority of the memory. The rest (float/int64s) are using 8 bytes, however these may also be reduced to 16 or 32.

In [13]:
def cols(col_names, data_type):
    '''
    Takes in list of column names and a datatype and generates a dictionary
    '''
    return {col_names[i]: data_type for i in range(len(col_names))}

In [19]:
def clean_data_types(df):
    '''
    Cleans datatypes for raw data
    '''
    df.columns = map(str.lower, df.columns)
    
    # String/text columns
    string_cols = ['cusip_id', 'cusip', 'prospectus_issuer_name']
    strings = cols(string_cols, 'string[pyarrow]')

    bool_cols = ['callable', 'make_whole', 'conv', 'junior']
    bools = cols(bool_cols, 'bool')

    category_cols = ['security_level', 'rating_spr', 'rating_mdy', 'standard', 'putop', 'industry_code']
    categories = cols(category_cols, 'category')
    try:
        df = df.drop(["sp_rat", "moody_rat", "rating", 'rating_date_spr', 'rating_date_mdy'], axis=1)
    except: pass
        
    # Float/Integer columns
    int8_cols = ['interest_frequency']
    int8s = cols(int8_cols, 'int8')

    df['sic_code'] = df['sic_code'].fillna(0)
    int16_cols = ['sic_code']
    int16s = cols(int16_cols, 'int16')

    int32_cols = ['volume', 'offering_amt', 'amount_outstanding', 'issue_id', 'issuer_id']
    int32s = cols(int32_cols, 'int32')

    float32_cols = ['price', 'offering_price', 'coupon', 'accrued_interest', 'tau',
                   'age', 'ytm', 'duration', 'tr_dirty_price', 'tr_ytm', 'cs']
    float32s = cols(float32_cols, 'float32')

    # Dates
    # formula to convert to datetime: df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    df['offering_date'] = df['offering_date'].fillna(0)
    date_cols = ['date', 'maturity', 'offering_date', 'month1']
    dates = cols(date_cols, 'int32')

    # Applies type changes
    types = strings | bools | categories | int8s | int16s | int32s | float32s | dates

    return df.astype(types)

In [192]:
df = clean_data_types(df)
# df.memory_usage(deep=True)

In [198]:
df.info()
# Memory usage decreased by about half

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   cusip_id                1000 non-null   string  
 1   date                    1000 non-null   int32   
 2   price                   1000 non-null   float32 
 3   volume                  1000 non-null   int32   
 4   issue_id                1000 non-null   int32   
 5   issuer_id               1000 non-null   int32   
 6   prospectus_issuer_name  1000 non-null   string  
 7   maturity                1000 non-null   int32   
 8   security_level          1000 non-null   category
 9   offering_amt            1000 non-null   int32   
 10  offering_date           1000 non-null   int32   
 11  offering_price          852 non-null    float32 
 12  interest_frequency      1000 non-null   int8    
 13  coupon                  1000 non-null   float32 
 14  standard                1

In [204]:
# Loads in whole data
data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")

C:\Users\sroma\AppData\Local\Temp\ipykernel_25248\3165405631.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")


14090424.827 KB


In [221]:
# Cleans data
cleaned_data = clean_data_types(data)

14090424.827 KB


In [ ]:
# Saves cleaned data
cleaned_data.to_csv('cleaned_data.csv')

In [222]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18833993 entries, 0 to 18833992
Data columns (total 40 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   cusip_id                object 
 1   date                    int64  
 2   price                   float64
 3   volume                  float64
 4   issue_id                int64  
 5   issuer_id               int64  
 6   prospectus_issuer_name  object 
 7   maturity                int64  
 8   security_level          object 
 9   offering_amt            float64
 10  offering_date           float64
 11  offering_price          float64
 12  interest_frequency      int64  
 13  coupon                  float64
 14  standard                int64  
 15  putop                   int64  
 16  conv                    int64  
 17  junior                  int64  
 18  cusip                   object 
 19  callable                object 
 20  make_whole              object 
 21  industry_code           int64

In [224]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18833993 entries, 0 to 18833992
Data columns (total 35 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   cusip_id                string  
 1   date                    int32   
 2   price                   float32 
 3   volume                  int32   
 4   issue_id                int32   
 5   issuer_id               int32   
 6   prospectus_issuer_name  string  
 7   maturity                int32   
 8   security_level          category
 9   offering_amt            int32   
 10  offering_date           int32   
 11  offering_price          float32 
 12  interest_frequency      int8    
 13  coupon                  float32 
 14  standard                category
 15  putop                   category
 16  conv                    bool    
 17  junior                  bool    
 18  cusip                   string  
 19  callable                bool    
 20  make_whole              bool    
 21  indust

# Aggregates Data to Monthly

In [2]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [132]:
# Loads in subset of cleaned data
with pd.read_csv("cleaned_data.csv", chunksize=100_000, index_col=0) as reader:
    data = reader.get_chunk()

# csv doesn't preserve data types
data = clean_data_types(data)

In [133]:
print(f''' With the subset of data there are: 
    - {len(data['issuer_id'].unique())} unique issuers 
    - {len(data['issue_id'].unique())} unique bonds  
    - {len(data['date'].sort_values().unique())} days in the sample
    ''')

 With the subset of data there are: 
    - 43 unique issuers 
    - 192 unique bonds  
    - 4913 days in the sample
    


## Sample Exclusion Criteria
Based on Bai, Bali & Wen (2019):

1. Remove bonds that are not listed or traded in the US public market, which include bonds issued through private placement, bonds issued under the 144A rule, bonds that do not trade in US dollars, and bond issuers not in the jurisdiction of the United States.
2. Remove bonds that are structured notes, mortgage backed or asset backed, agency backed, or equity linked.
3. Remove convertible bonds since this option feature distorts the return calculation and makes it impossible to compare the returns of convertible and nonconvertible bonds.
4. Remove bonds that trade under 5 or above 1000. 
5. Remove bonds that have a floating coupon rate, which means the sample comprises only bonds with a fixed or zero coupon. This rule is applied based on the consideration of the accuracy in bond return calculation, given the challenge in tracking a floating-coupon bond’s cash flows.
6. Remove bonds that have less than one year to maturity. This rule is applied to all major corporate bond indices such as the Barclays Capital Corporate Bond Index, the Bank of America Merrill Lynch Corporate Master Index, and the Citi Fixed Income Indices. If a bond has less than one year to maturity, it will be delisted from major bond indices; hence, index-tracking investors will change their holding positions. This operation will distort the return calculation for bonds with less than one year to maturity; thus, we remove them from our sample.
7. For intraday data, we also eliminate bond transactions that are labeled as when-issued, locked-in, or have special sales conditions, and that have more than a twoday settlement.
8. Remove transaction records that are canceled and adjust records that are subsequently corrected or reversed.
9. Remove transaction records that have trading volume less than $10,000

### 3. Convertible Bonds

In [156]:
def filter_bond_convertible(data):
    '''
    Function which filters out convertible bonds.
    '''
    
    return data[data['conv'] == False]

In [157]:
data = filter_bond_convertible(data)

### 4. Bonds with irregular trading prices

In [162]:
def filter_bond_prices(data):
    '''
    Function which filters out bonds that trade below $5 or above $1000.
    '''
    
    return data[(data['price'].gt(5)) | (data['price'].lt(1000))]

In [163]:
data = filter_bond_prices(data)

### 6. Bonds with less than 1-year to maturity
Perhaps worthwhile to keep seperate as a robustness check?

In [134]:
def filter_bond_maturity(data):
    '''
    Function which filters out bonds with less than 1-year to maturity.
    Saves bonds with less than 1-year to maturity in a seperate file.
    '''
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
    data['maturity'] = pd.to_datetime(data['maturity'], format='%Y%m%d')

    data = data.sort_values(by='date')
    data['days_to_maturity'] = data['maturity'] - data['date']

    # Filters out bonds with less than 1-year to maturity
    data_one_year_bonds = data[data['days_to_maturity'].le('365 days')]
    data_one_year_bonds.to_csv('bonds_1_year_maturity.csv')
    
    return data[data['days_to_maturity'].gt('365 days')]

In [135]:
data = filter_bond_maturity(data)

In [137]:
data.shape

(96721, 36)

### 9. Bonds with small trading volumes

In [143]:
def filter_bond_volume(data):
    '''
    Function which filters daily volumes of less than $10,000.
    '''
    
    return data[data['volume'].gt(10000)]

In [144]:
data = filter_bond_volume(data)

In [146]:
data.shape

(95529, 36)

### Monthly Return Aggregation

The monthly corporate bond return at time $t$ is computed as 

<font size=6>
$
r_{i,t} = \frac{P_{i,t} + AI_{i,t} + C_{i,t}}{P_{i,t−1} + AI_{i,t−1}} − 1
$
</font>

Where $P_{i,t}$ is month $t$'s bond clean price, $AI_{i,t}$ is the accrued-interest, and $C_{i,t}$ is the coupon.

Excess returns are computed by deducting the 1-month Treasury bill rate.

Further, following Bai, Bali & Wen (2019), returns can be computed as either end of month t relative to end of month t-1, or end of month t to begining of month t. Where the end/beginning of the month is defined with a 5 day window. The date which is closest to the last/firsy trading day is used. If both such return calculations are available, then the end-to-end is prefered.

#### End-to-End Method

In [234]:
# data

In [374]:
def end_month_returns(data):
    '''
    Function to compute monthly returns using the end-to-end method.
    '''
    # Ensures that data is sorted in chronological order
    data = data.sort_values(by='date')
    
    # Gets 5 days from the end of the month
    data['month_end'] = data['date'] + MonthEnd(0) - pd.Timedelta(days=5)
    
    # Groups by issue id and year-month: returns the last observation
    data = data.groupby(['issue_id', test2['date'].dt.year, test2['date'].dt.month]).tail(1).copy()
    
    # Creates the P + AI + C for time t
    data['full_price_t'] = data['price'] + data['accrued_interest'] + data['coupon']
    data['price_t'] = data['price'] + data['accrued_interest']
    
    # Lags P + AI
    data['price_t1'] = data.groupby('issue_id')['price_t'].shift(1)
    
    # Computes the return only if the transaction date is within 5 days of end of month
    data['return'] = np.where(~(data['date'] < data['month_end']), 
                               (data['full_price_t'] / data['price_t']) - 1, np.nan)
    
    return data 

In [375]:
end_data = end_month_returns(data)

In [ ]:
data[(data['issue_id'] == 55723) | (data['issue_id'] == 76070)]

In [368]:
test2 = data[(data['issue_id'] == 55723) | (data['issue_id'] == 76070)].copy()
last2 = test2.groupby(['issue_id', test2['date'].dt.year, test2['date'].dt.month]).tail(1).copy()
last2['month_end'] = last2['date'] + MonthEnd(0) - pd.Timedelta(days=5)
last2['price_l1'] = last2.groupby('issue_id')['price'].shift(1)
last2['return'] = np.where(~(last2['date'] < last2['month_end']), (last2['price'] / last2['price_l1']) - 1, np.nan)

In [369]:
last2

,cusip_id,date,price,volume,issue_id,issuer_id,prospectus_issuer_name,maturity,security_level,offering_amt,...,age,ytm,duration,tr_dirty_price,tr_ytm,cs,days_to_maturity,month_end,price_l1,return
21396,001031AC7,2002-07-26,97.500000,1200000,76070,20591,AEP INDS INC,2007-11-15,SENS,200000,...,4.320548,10.494145,4.158610,132.368637,3.528975,6.965170,1938 days,2002-07-26,NaN,NaN
6712,00077QAG5,2002-07-31,111.155998,135000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,5.139726,4.549879,4.219957,116.946976,3.505619,1.044260,1783 days,2002-07-26,NaN,NaN
21397,001031AC7,2002-08-05,98.000000,100000,76070,20591,AEP INDS INC,2007-11-15,SENS,200000,...,4.347945,10.368834,4.137018,133.922668,3.278418,7.090416,1928 days,2002-08-26,97.500000,NaN
6732,00077QAG5,2002-08-30,111.000000,15000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,5.221918,4.546115,4.136686,118.938469,3.161422,1.384694,1753 days,2002-08-26,111.155998,-0.001403
21403,001031AC7,2002-09-25,98.062500,20000,76070,20591,AEP INDS INC,2007-11-15,SENS,200000,...,4.487671,10.364343,3.998251,136.685776,2.892019,7.472324,1877 days,2002-09-25,98.000000,0.000638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7259,00077QAG5,2006-02-28,102.173225,155000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,8.723288,5.362915,1.249673,104.382256,4.732125,0.630790,475 days,2006-02-23,102.889557,-0.006962
7274,00077QAG5,2006-03-31,102.004257,481000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,8.808219,5.390944,1.166329,104.624817,4.881485,0.509459,444 days,2006-03-26,102.173225,-0.001654
7288,00077QAG5,2006-04-28,101.798996,45000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,8.884932,5.465383,1.088522,104.956177,4.931460,0.533922,416 days,2006-04-25,102.004257,-0.002012
7305,00077QAG5,2006-05-31,101.723068,3030000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,8.975343,5.410569,0.999655,105.247841,5.103238,0.307332,383 days,2006-05-26,101.798996,-0.000746


In [241]:
test = data[data['issue_id'] == 55723].copy()

In [261]:
last = test.groupby([test['date'].dt.year, test['date'].dt.month]).tail(1).copy()


In [262]:
last['month_end'] = last['date'] + MonthEnd(0) - pd.Timedelta(days=5)

In [284]:
# Return only computed if last price is within 5 days of last day of month
last['return'] = np.where(~(last['date'] < last['month_end']), (last['price'] / last['price'].shift(1)) - 1, np.nan)

In [372]:
last[['price', 'accrued_interest', 'tr_dirty_price', 'coupon']]

,price,accrued_interest,tr_dirty_price,coupon
6712,111.155998,0.837090,116.946976,7.125
6732,111.000000,1.421107,118.938469,7.125
6746,113.926003,2.024590,121.981346,7.125
6761,111.625000,2.628074,121.710030,7.125
6773,112.055977,3.134221,120.536217,7.125
6783,113.232002,0.234890,119.374222,7.125
6796,112.875000,0.861264,118.770035,7.125
6809,115.228996,1.409341,120.548424,7.125
6823,114.969292,1.957418,120.122269,7.125
6839,115.526474,2.603365,120.775368,7.125


#### Start-to-End Method

In [ ]:
# first = test.groupby([test['date'].dt.year, test['date'].dt.month]).head(1)
# last = test.groupby([test['date'].dt.year, test['date'].dt.month]).tail(1).copy()